### scrapy
- 비동기 방식으로 데이터를 수집 > 데이터 수집 속도가 빠름

In [2]:
import scrapy, requests
from scrapy.http import TextResponse

In [3]:
# Gmarket 베스트 200 상품 데이터 수집

In [4]:
# 1. 스크레피 프로젝트 생성 

In [8]:
!scrapy startproject gmarket
# 마지막에는 프로젝트 이름

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [9]:
!tree gmarket /f

폴더 PATH의 목록입니다.
볼륨 일련 번호가 000000FA 7C6C:FC35입니다.
C:\USERS\USER\GMARKET
│  scrapy.cfg
│  
└─gmarket
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


In [ ]:
# items.py : 수집할 데이터의 구조 정의
# middlewares.py : 데이터를 수집할 때 headers 정보와 같은 내용을 설정
# pipelines.py : 데이터를 수집한 후에 코드 실행 정의
# settings.py : 크롤링 설정 : 크롤링 시간 텀, robots.txt 규칙
# spiders : 디렉토리 : 크롤링 절차 정의

In [5]:
# 2. xpath 찾기 : 링크, 상세 페이지의 데이터

In [ ]:
url = "http://corners.gmarket.co.kr/Bestsellers"

In [10]:
# 링크 데이터 수집

In [14]:
request = requests.get("http://corners.gmarket.co.kr/Bestsellers")
response = TextResponse(request.url, body=request.text, encoding="utf-8")

In [17]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
links[:2]

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953126299881366',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953126299881366']

In [11]:
# 상세 페이지 데이터 수집

In [19]:
link = links[0]
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding="utf-8")

In [20]:
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()
title, price

('브라운 면도기 세정액 CCR5+1 ', '25,030')

In [6]:
# 3. items.py : 코드 작성 : model(데이터의 구조: 수집할 데이터의 컬럼을 정의)

In [26]:
%%writefile gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [ ]:
# 4. spider.py: 코드 작성 : 크롤링 절차 정의

In [27]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Overwriting gmarket/gmarket/spiders/spider.py


In [ ]:
# 5. 스크레피 프로젝트 실행
# gmarket 디렉토리 
# -o : ouput
# scrapy crawl GMB -o items.csv

In [25]:
%pwd

'C:\\Users\\User'

In [ ]:
# C:\\Users\\User\\Gmarket 으로하면 unknown 에러 해결됨

In [29]:
pd.read_csv("gmarket/items.csv")

,link,price,title
0,http://item.gmarket.co.kr/Item?goodscode=25196...,"25,030",브라운 면도기 세정액 CCR5+1
1,http://item.gmarket.co.kr/Item?goodscode=16659...,"15,900",무료반품/10+12%/런닝 5매/남성/남자/여성/여자/아동/속옷/모시/인견/텐셀/쿨...
2,http://item.gmarket.co.kr/Item?goodscode=19279...,"23,900",양꼬치 양깍두기 양고기 1kg + 쯔란1봉
3,http://item.gmarket.co.kr/Item?goodscode=22825...,"5,900",풀문 여름 시즌오프 버킷햇/레그워머/양말/왕리본/집게핀
4,http://item.gmarket.co.kr/Item?goodscode=22474...,"3,900",10%할인+사은품_책속에서 여름나기 인기도서 260종_무료배송
5,http://item.gmarket.co.kr/Item?goodscode=20965...,"109,000",BLDC 입체회전 써큘레이터 SIF-DPNW90 ---시즌오프 특가---
6,http://item.gmarket.co.kr/Item?goodscode=20396...,"49,900",1+1 2개 무선 선풍기 탁상용 미니 휴대용 캠핑용 벽걸이 사무실 선물용 가정용
7,http://item.gmarket.co.kr/Item?goodscode=16359...,"9,900",아삭아삭 미니 오이 20개 /2개주문시 50개발송
8,http://item.gmarket.co.kr/Item?goodscode=24572...,"12,000",레고 닌자고 브릭마스터 외 6종 선택구매
9,http://item.gmarket.co.kr/Item?goodscode=20361...,"10,000",무료반품 15% 할인쿠폰 마르조 오나라 단하루특가
